In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
import random
import string
import nltk
import spacy
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import markovify

from sqlalchemy import create_engine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import gutenberg

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement

import warnings

warnings.filterwarnings(action="ignore")

<IPython.core.display.Javascript object>

In [5]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "twitter_sentiment"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

tweets_df = pd.read_sql_query("select * from twitter", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


tweets_df.head(10)

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)
5,5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,None,jnardino,None,0,@VirginAmerica seriously would pay $30 a fligh...,None,2015-02-24 11:14:33 -0800,None,Pacific Time (US & Canada)
6,6,570300616901320704,positive,0.6745,None,0.0000,Virgin America,None,cjmcginnis,None,0,"@VirginAmerica yes, nearly every time I fly VX...",None,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,7,570300248553349120,neutral,0.6340,None,NaN,Virgin America,None,pilot,None,0,@VirginAmerica Really missed a prime opportuni...,None,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,8,570299953286942721,positive,0.6559,None,NaN,Virgin America,None,dhepburn,None,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",None,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,9,570295459631263746,positive,1.0000,None,NaN,Virgin America,None,YupitsTate,None,0,"@VirginAmerica it was amazing, and arrived an ...",None,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


<IPython.core.display.Javascript object>

In [7]:
pos_tweets_df = tweets_df[tweets_df["airline_sentiment"] == "positive"]
neg_tweets_df = tweets_df[tweets_df["airline_sentiment"] == "negative"]
nlp = spacy.load("en", disable=["parser", "ner"])
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.max_length = 20000000
pos_tweets = " ".join(pos_tweets_df["text"])
neg_tweets = " ".join(neg_tweets_df["text"])

pos_tweets_doc = nlp(pos_tweets)
neg_tweets_doc = nlp(neg_tweets)

<IPython.core.display.Javascript object>

In [9]:
# Group into sentences
# Use the sentences that have more than one character
pos_tweets_sents = " ".join(
    [sent.text for sent in pos_tweets_doc.sents if len(sent.text) > 1]
)
neg_tweets_sents = " ".join(
    [sent.text for sent in neg_tweets_doc.sents if len(sent.text) > 1]
)


<IPython.core.display.Javascript object>

In [10]:
pos_tweet_generator = markovify.Text(pos_tweets_sents, state_size=3)
neg_tweet_generator = markovify.Text(neg_tweets_sents, state_size=3)

<IPython.core.display.Javascript object>

In [13]:
# Three randomly generated sentences
for i in range(3):
    print(pos_tweet_generator.make_sentence())
print("-----------------------")
for i in range(3):

    print(neg_tweet_generator.make_sentence())

print("-----------------------")
print("-----------------------")

# Three randomly generated sentences of no more than 100 characters
for i in range(3):

    print(pos_tweet_generator.make_short_sentence(100))
print("-----------------------")
for i in range(3):
    print(neg_tweet_generator.make_short_sentence(100))

@USAirways thanks for the effort!
None
Looking forward to my global first class flight on your 777 to FRA.
-----------------------
@AmericanAir Any info for Cancelled Flightled flight was useless to my assistant's trip.
@AmericanAir Asked for a waiver of a change fee request from a month ago.
Every flight with you guys.
-----------------------
-----------------------
@SouthwestAir crew on flight 475!
@AmericanAir Thanks for the help!
@united is my favorite airline!
-----------------------
Now is Late Flight due to mechanical.
@united Don't ask me to be loyal if don't let me carry on my bag that had all my ski gear in it.
Does that mean I have to wait a whole day to get a refund?


<IPython.core.display.Javascript object>

In [14]:
class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

<IPython.core.display.Javascript object>

In [15]:
pos_tweet_generator = POSifiedText(pos_tweets_sents, state_size=3)
neg_tweet_generator = POSifiedText(neg_tweets_sents, state_size=3)

<IPython.core.display.Javascript object>

In [19]:
# Three randomly generated sentences
for i in range(3):
    print(pos_tweet_generator.make_sentence())
print("-----------------------")
for i in range(3):

    print(neg_tweet_generator.make_sentence())

print("-----------------------")
print("-----------------------")

# Three randomly generated sentences of no more than 100 characters
for i in range(3):

    print(pos_tweet_generator.make_short_sentence(100))
print("-----------------------")
for i in range(3):
    print(neg_tweet_generator.make_short_sentence(100))

@united Got me home amid snow & amp ; the wifi is appreciated .
Thank you ! ! Yayayay ! ! @AmericanAir thx , just sent the DM @AmericanAir Kudos to the captain and crew for handling fustrated people while dealing with the weather !
None
-----------------------
@AmericanAir Now your site is down ?
None
@USAirways hour 4 at the gate for the broken jetway .
-----------------------
-----------------------
Excited to fly with you guys .
Her flight leaves at 2 but she 's arriving to the airport the rebooked ticket was refunded .
Y’all have a good one ! ! @AmericanAir Eventually the call got through .
-----------------------
@united Stuck in Houston because you ca n't even describe .
They 're flexible & amp ; asked if I wanted to .
@JetBlue trying to charge me an additional $ 200 fee .


<IPython.core.display.Javascript object>